In [34]:
import os
import time
import math
import arxiv
import requests
import numpy as np
from PIL import Image
from io import BytesIO
from openai import OpenAI
from pydub import AudioSegment
from moviepy.editor import concatenate, ImageClip, VideoFileClip, AudioFileClip, CompositeAudioClip

In [35]:
# Construct the default API client.
arxivClient = arxiv.Client()

# Search for the paper with ID "1605.08386v1"
search_by_id = arxiv.Search(id_list=["2401.01871v1"])
# Reuse client to fetch the paper, then print its title.
first_result = next(arxivClient.results(search_by_id))
researchPaperTitle = first_result.title
researchPaperSummary = first_result.summary

In [36]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

assistant = client.beta.assistants.create(
  name="Researcher",
  instructions="""You are a research scientist assistant, skilled in explaining complex academic 
  concepts with simplicity and creative flair. When given the title and abstract from any academic 
  research paper, you are able to easily break down research paper abstracts into several parts 
  to help others understand what the research paper is all about. I will be giving you a abstract from a
  research paper and you need to perform the following operations:
  1. Rephrase the following text to eliminate the LaTex from it without changing its meaning.
  After this, present the result obtained from previous operation in 10 points. Keep it academic. DO NOT CREATE ANY NEW INFORMATION OR HALLUCINATE
  2. Only output the 10 points generated from the previous operation, nothing else""",
  model="gpt-3.5-turbo-1106",
)
thread = client.beta.threads.create()

In [41]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=
    """
    Title: """ + researchPaperTitle + """\n"""
    """Abstract: """ + researchPaperSummary
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [44]:
while True:
    sleep_time = 3
    print(f"Sleeping for {sleep_time} seconds")
    time.sleep(sleep_time)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print("Current Status: ", run.status)
    if run.status != 'in_progress':
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        print(messages.data[0].content[0].text.value)
        break

Sleeping for 3 seconds
Current Status:  failed


In [19]:
text = messages.data[0].content[0].text.value
imagePrompts = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": """You are a prompt generator assistant for my DALL-E text to image model, skilled in generating suitable and correct
     prompts for any text given you. When given a series of texts, you have to generate a DALL-E image prompt 
     for each of the given points in the text.
     The format for your response for each prompt is fixed and given below:
     Prompt: The prompt text beginning with 'Generate an image'"""},
    {"role": "user", "content": text}
  ]
)

print(imagePrompts.choices[0].message.content)

Prompt 1: Generate an image of WISE J224607.6-052634.9 (W2246-0526), the most luminous obscured quasar known, located at a redshift of 4.601.

Prompt 2: Generate an image depicting a heavily obscured supermassive black hole in W2246-0526, which is likely accreting above the Eddington limit.

Prompt 3: Generate an image representing the Atacama Large Millimeter/submillimeter Array (ALMA) being used to observe the far-infrared (FIR) fine-structure emission lines of W2246-0526.

Prompt 4: Generate an image showing the emission lines observed in W2246-0526, including [OI] at 63 micrometers, [OIII] at 88 micrometers, [NII] at 122 micrometers, [OI] at 145 micrometers, [CII] at 158 micrometers, [NII] at 205 micrometers, [CI] at 370 micrometers, and [CI] at 609 micrometers.

Prompt 5: Generate an image illustrating the necessity for high hydrogen density, extinction, extreme ionization, and a high X-ray to UV ratio to reproduce the observed nuclear line ratios in W2246-0526.

Prompt 6: Generat

In [20]:
import re

allImagePrompts = []
# Your input text
input_text = imagePrompts.choices[0].message.content # Replace with your actual text

# Regular expression to match lines starting with 'Generate'
pattern = r"Generate.*"

# Find all matches
matches = re.findall(pattern, input_text)

# Printing or processing the matches
for match in matches:
    allImagePrompts.append(match)


In [21]:
text = messages.data[0].content[0].text.value
allPoints = []
for i, t in enumerate(text.split('\n')):
    if t == '':
        continue
    elif t[:2] == '10':
        allPoints.append(t[4:])
    else:
        allPoints.append(t[3:])
for p in allPoints:
    print(p)

The research paper discusses WISE J224607.6-052634.9 (W2246-0526), which is a hot dust-obscured galaxy located at a redshift of 4.601, and is currently the most luminous obscured quasar known.
W2246-0526 contains a heavily obscured supermassive black hole that is likely accreting above the Eddington limit.
The study presents observations conducted using the Atacama Large Millimeter/submillimeter Array (ALMA) in seven bands, specifically focusing on the brightest far-infrared (FIR) fine-structure emission lines of this galaxy.
The observed emission lines include [OI] at 63 micrometers, [OIII] at 88 micrometers, [NII] at 122 micrometers, [OI] at 145 micrometers, [CII] at 158 micrometers, [NII] at 205 micrometers, [CI] at 370 micrometers, and [CI] at 609 micrometers.
A comparison of the data with a large grid of Cloudy radiative transfer models reveals the necessity for high hydrogen density, extinction, extreme ionization, and a high X-ray to UV ratio to reproduce the observed nuclear li

In [22]:
allImageURLs = []
for i, promptText in enumerate(allImagePrompts):
    response = client.images.generate(
        model="dall-e-2",
        prompt=promptText,
        size="1024x1024",
        quality="standard",
        n=1,
    )
    print(response)
    allImageURLs.append(response.data[0].url)
    if i == 4:
        sleep_time = 30
        print(f"Sleeping for {sleep_time} seconds")
        time.sleep(sleep_time)

ImagesResponse(created=1704659149, data=[Image(b64_json=None, revised_prompt=None, url='https://oaidalleapiprodscus.blob.core.windows.net/private/org-jaDt9ccyHF30RXiXHd6Uw3hz/user-katRTQBktBWTA9sOFdAa4QCz/img-qTFWhha6PNFi9ckv1d0mHOUj.png?st=2024-01-07T19%3A25%3A49Z&se=2024-01-07T21%3A25%3A49Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-01-07T15%3A52%3A09Z&ske=2024-01-08T15%3A52%3A09Z&sks=b&skv=2021-08-06&sig=BliUX%2BZ/LOKlBbQ3TLfKTuYBZy%2BXigUTXmjKO7Gt8ME%3D')])
ImagesResponse(created=1704659158, data=[Image(b64_json=None, revised_prompt=None, url='https://oaidalleapiprodscus.blob.core.windows.net/private/org-jaDt9ccyHF30RXiXHd6Uw3hz/user-katRTQBktBWTA9sOFdAa4QCz/img-RMsFGQ0ZIwYyK5lSCrFYQnWz.png?st=2024-01-07T19%3A25%3A58Z&se=2024-01-07T21%3A25%3A58Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&

In [23]:
allImages = []
for url in allImageURLs:
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    allImages.append(img)

In [24]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": """You are a research scientist assistant, skilled in explaining complex academic \\
    concepts with simplicity and creative flair. When given the title and key points from any academic 
    research paper, you are able to create a short voiceover script of about 1 minute 30 seconds in the style of David Attenborough. You must 
    only provide the content for the voiceover, nothing else in your response like background music or the name of David David Attenborough. 
    You must generate the script with around 300 words only."""},
    {"role": "user", "content": """Title: """ + researchPaperTitle + """ \n"""
     """Key Points: """ + text}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="In the vast expanse of the universe, there exists a galaxy like no other. Its name - WISE J224607.6-052634.9, or as we shall call it, W2246-0526. This remarkable galaxy, located at a redshift of 4.601, shines brighter than any other obscured quasar known to us.\n\nDeep within W2246-0526 lies a hidden secret. A supermassive black hole, heavily obscured, silently accreting at a rate that defies limits. This galaxy, my dear viewers, is a true benchmark for extreme conditions.\n\nTo unravel its mysteries, scientists turned their gaze towards the Atacama Large Millimeter/submillimeter Array, or ALMA for short. Armed with this powerful tool, they peered into the depths of W2246-0526 across seven bands.\n\nWhat they discovered were the brightest far-infrared fine-structure emission lines of this astonishing galaxy. Lines such as [OI], [OIII], [NII], [CII], [CI], and more, each revealing a tantalizing piece of the puzzle.\n\nComparing the data with a multitude of

In [25]:
speech_file_path = "./speech.mp3"
response = client.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input=completion.choices[0].message.content
)

response.stream_to_file(speech_file_path)

In [26]:
# Load your MP3 file
audio = AudioSegment.from_mp3("./speech.mp3")

# Calculate the duration in milliseconds
duration_ms = len(audio)

# Convert the duration to seconds
duration_seconds = duration_ms / 1000

print(f"Duration: {duration_seconds} seconds")

Duration: 141.792 seconds


In [27]:
clips = []

for i in range(int(duration_seconds // 30)):
  for image in allImages:
    clips.append(ImageClip(np.array(image)).set_duration(3))

for i in range(math.ceil((duration_seconds % 30) / 3)):
  clips.append(ImageClip(np.array(allImages[i])).set_duration(3))

video = concatenate(clips, method="compose")
video.write_videofile('test1.mp4', fps=24)

Moviepy - Building video test1.mp4.
Moviepy - Writing video test1.mp4



Moviepy - Done !
Moviepy - video ready test1.mp4


In [28]:
videoclip = VideoFileClip("./test1.mp4")
audioclip = AudioFileClip("./speech.mp3")

new_audioclip = CompositeAudioClip([audioclip])
videoclip.audio = new_audioclip
videoclip.write_videofile("output.mp4")

Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
